In [1]:
import os

import pandas as pd

In [2]:
ROOT_PATH = "/home/ay/data/DATA/1-model_save/0-Audio"

In [3]:
MODELs = [
    "LCNN",
    "RawNet2",
    "Wave2Vec2",
    "WaveLM",
    "LibriSeVoc",
    "AudioClip",
    "Wav2Clip",
    "AASIST",
    "Ours",
]

In [4]:
for model in ["LCNN", "RawNet2", "Wave2Vec2", "WaveLM", "LibriSeVoc"]:
    for task in [
        "LibriSeVoc_inner",
        "LibriSeVoc_cross_method",
        "LibriSeVoc_cross_dataset",
        "wavefake_inner",
        "wavefake_cross_lang",
        "wavefake_cross_method",
    ]:
        pass

# Tasks

Inner Evaluation:
* `LibriSeVoc_inner`：使用6个vocoder和real进行训练和测试
* `wavefake_inner`：在LJSpeech上，使用7个vocoder和real进行训练和测试

Cross Method:
* `LibriSeVoc_cross_method`：使用2个vocoder和real进行训练，在剩下4个vocder上测试
* `wavefake_cross_method`: 使用2个vocoder和real进行训练，在剩下5个vocder上训练

Cross dataset:
* `LibriSeVoc_cross_dataset`：在LibriSeVoc的所有6个vocder和real上进行训练，在wavefake上的每一个vocoder进行测试。

Cross Language:
* `wavefake_cross_lang`：在LJSpeech上，使用2个vocoder和real进行训练，在JSUT上的两个vocoder和real进行测试。

## Inner Evaluation

In [5]:
DATA = []
for task in [
    "LibriSeVoc_inner",
    "wavefake_inner",
]:
    for model in MODELs:
        save_path = f"{ROOT_PATH}/{model}/{task}/version_0"
        csv_path = os.path.join(save_path, "metrics.csv")
        data = pd.read_csv(csv_path)

        data = data[["test-acc", "test-auc", "test-eer"]]
        res = dict(data.iloc[-1])
        res["res"] = "{:.4f}/{:.4f}/{:.2f}".format(
            res["test-acc"], res["test-auc"], res["test-eer"] * 100
        )
        res["model"] = model
        res["dataset"] = task.split("_")[0]
        DATA.append(res)
data = pd.DataFrame(DATA)

In [6]:
print(data.pivot(index="model", columns="dataset", values="res").style.to_latex())

\begin{tabular}{lll}
dataset & LibriSeVoc & wavefake \\
model &  &  \\
AASIST & 0.9866/0.9866/0.39 & 0.9912/0.9912/0.58 \\
AudioClip & 0.9665/0.9665/3.40 & 0.9894/0.9588/1.00 \\
LCNN & 0.9946/0.9946/0.49 & 0.9886/0.9881/1.16 \\
LibriSeVoc & 0.9915/0.9915/0.74 & 0.9693/0.9693/3.05 \\
Ours & 0.9858/0.9909/0.49 & 0.5569/0.7468/0.71 \\
RawNet2 & 0.9844/0.9844/1.54 & 0.9621/0.9637/3.55 \\
Wav2Clip & 0.9325/0.9325/8.33 & 0.9991/0.9984/0.13 \\
Wave2Vec2 & 0.9974/0.9974/0.27 & 0.9980/0.9954/0.33 \\
WaveLM & 0.9963/0.9963/0.34 & 0.9875/0.9873/1.26 \\
\end{tabular}



## Cross Method

In [7]:
DATA = []
for task in [
    "LibriSeVoc_cross_method",
    "wavefake_cross_method",
]:
    for model in MODELs:
        save_path = f"{ROOT_PATH}/{model}/{task}/version_0"
        csv_path = os.path.join(save_path, "metrics.csv")
        data = pd.read_csv(csv_path)
        try:
            data = data[["test-acc", "test-auc", "test-eer"]]
        except KeyError:
            continue
        res = data.dropna().reset_index(drop=True)
        res["res"] = res.apply(
            lambda x: "{:.3f}/{:.2f}".format(x["test-auc"], x["test-eer"] * 100),
            axis=1,
        )
        res["model"] = model
        res["dataset"] = task.split("_")[0]
        res["method"] = [1, 2, 3, 5] if task.startswith("Lib") else [1, 2, 3, 4, 6]
        res = res[["dataset", "model", "method", "res"]]
        DATA.append(res)
data = pd.concat(DATA)

In [8]:
t = data.pivot(index="model", columns=["dataset", "method"], values="res")
t

dataset      LibriSeVoc                                            wavefake  \
method                1            2            3            5            1   
model                                                                         
AASIST      0.522/25.63  0.505/27.39  0.513/29.59  0.581/17.15  0.500/39.85   
AudioClip   0.663/19.79  0.580/22.75  0.536/33.40  0.789/11.93  0.500/49.44   
LCNN        0.626/11.97  0.569/17.49  0.520/26.33  0.527/23.69  0.501/35.34   
LibriSeVoc  0.520/35.33  0.504/41.07  0.502/46.69  0.535/29.99  0.500/47.18   
Ours        0.898/10.03  0.778/16.66  0.723/22.86   0.930/6.97  0.500/48.28   
RawNet2     0.550/27.83  0.512/31.81  0.504/40.55  0.542/27.91  0.500/46.34   
Wav2Clip     0.592/8.38  0.503/35.78  0.503/43.17  0.531/21.60  0.500/34.61   
Wave2Vec2   0.529/15.01   0.546/7.84  0.517/21.55  0.501/35.41  0.500/57.10   
WaveLM      0.540/18.48  0.547/14.16  0.515/29.19  0.501/37.76  0.500/48.63   

dataset                                                         
method                2            3            4            6  
model                                                           
AASIST      0.500/44.42  0.500/45.52  0.500/48.64  0.501/27.71  
AudioClip   0.500/44.84  0.500/48.42  0.500/43.30  0.500/49.76  
LCNN        0.504/31.26  0.503/33.39  0.503/32.30  0.500/45.84  
LibriSeVoc  0.500/49.56  0.500/50.15  0.500/50.92  0.500/46.37  
Ours        0.500/46.39  0.500/46.07  0.500/41.59  0.500/39.18  
RawNet2     0.500/50.23  0.500/49.60  0.500/51.11  0.500/44.96  
Wav2Clip    0.501/19.25  0.502/16.60  0.500/24.24  0.500/29.96  
Wave2Vec2   0.500/42.31  0.500/45.96  0.500/46.87  0.537/12.94  
WaveLM      0.500/35.73  0.500/42.02  0.500/48.02  0.507/14.66

In [31]:
print(t.style.to_latex(column_format="lccccc", hrules=True))

\begin{tabular}{lccccc}
\toprule
dataset & \multicolumn{4}{r}{LibriSeVoc} & \multicolumn{5}{r}{wavefake} \\
method & 1 & 2 & 3 & 5 & 1 & 2 & 3 & 4 & 6 \\
model &  &  &  &  &  &  &  &  &  \\
\midrule
AASIST & 0.522/25.63 & 0.505/27.39 & 0.513/29.59 & 0.581/17.15 & 0.500/39.85 & 0.500/44.42 & 0.500/45.52 & 0.500/48.64 & 0.501/27.71 \\
AudioClip & 0.663/19.79 & 0.580/22.75 & 0.536/33.40 & 0.789/11.93 & 0.500/49.44 & 0.500/44.84 & 0.500/48.42 & 0.500/43.30 & 0.500/49.76 \\
LCNN & 0.626/11.97 & 0.569/17.49 & 0.520/26.33 & 0.527/23.69 & 0.501/35.34 & 0.504/31.26 & 0.503/33.39 & 0.503/32.30 & 0.500/45.84 \\
LibriSeVoc & 0.520/35.33 & 0.504/41.07 & 0.502/46.69 & 0.535/29.99 & 0.500/47.18 & 0.500/49.56 & 0.500/50.15 & 0.500/50.92 & 0.500/46.37 \\
RawNet2 & 0.550/27.83 & 0.512/31.81 & 0.504/40.55 & 0.542/27.91 & 0.500/46.34 & 0.500/50.23 & 0.500/49.60 & 0.500/51.11 & 0.500/44.96 \\
Wav2Clip & 0.592/8.38 & 0.503/35.78 & 0.503/43.17 & 0.531/21.60 & 0.500/34.61 & 0.501/19.25 & 0.502/16.60 & 0.500/2

## Cross Dataset

In [9]:
DATA = []
for task in [
    "LibriSeVoc_cross_dataset",
]:
    for model in MODELs:
        save_path = f"{ROOT_PATH}/{model}/{task}/version_0"
        csv_path = os.path.join(save_path, "metrics.csv")
        data = pd.read_csv(csv_path)
        try:
            data = data[["test-acc", "test-auc", "test-eer"]]
        except KeyError:
            continue
        res = data.dropna().reset_index(drop=True)
        res["res"] = res.apply(
            lambda x: "{:.4f}/{:.2f}".format(x["test-auc"], x["test-eer"] * 100),
            axis=1,
        )
        res["model"] = model
        res["method"] = [0, 1, 2, 3, 4, 5, 6]
        res = res[["model", "method", "res"]]
        DATA.append(res)
data = pd.concat(DATA)

In [10]:
t = data.pivot(index="model", columns="method", values="res")
t

method,0,1,2,3,4,5,6
model,,,,,,,
AASIST,0.8194/0.17,0.5002/10.56,0.5037/4.76,0.5001/15.78,0.5012/13.47,0.6326/0.56,0.7113/0.24
AudioClip,0.5633/11.41,0.5132/32.34,0.5577/12.50,0.5232/25.23,0.5397/19.48,0.5210/25.79,0.6856/4.95
LCNN,0.9062/0.47,0.5135/12.93,0.5111/12.53,0.5025/20.40,0.5033/18.88,0.9213/0.40,0.6898/2.27
LibriSeVoc,0.5369/17.40,0.4999/55.16,0.5002/49.89,0.4999/51.82,0.5001/49.79,0.5007/42.28,0.6653/3.53
Ours,0.5000/17.21,0.5001/32.60,0.5000/32.40,0.5000/42.73,0.5000/31.77,0.5000/17.51,0.5651/1.28
RawNet2,0.6991/4.19,0.5008/33.67,0.5049/23.08,0.5003/40.52,0.5015/31.79,0.5301/15.51,0.9742/0.39
Wav2Clip,0.5047/1.02,0.5000/12.83,0.5000/10.73,0.5000/9.92,0.5000/8.59,0.7363/0.26,0.5010/1.43
Wave2Vec2,0.5156/19.27,0.5008/35.84,0.5027/28.60,0.5001/42.81,0.5010/36.15,0.5040/26.69,0.6266/8.45
WaveLM,0.7255/10.97,0.5046/41.82,0.5418/25.21,0.5025/43.82,0.5161/35.01,0.5905/19.82,0.9642/2.40


In [10]:
print(t.style.to_latex(column_format="lccccccc", hrules=True))

\begin{tabular}{lccccccc}
\toprule
method & 0 & 1 & 2 & 3 & 4 & 5 & 6 \\
model &  &  &  &  &  &  &  \\
\midrule
AudioClip & 0.5633/11.41 & 0.5132/32.34 & 0.5577/12.50 & 0.5232/25.23 & 0.5397/19.48 & 0.5210/25.79 & 0.6856/4.95 \\
LCNN & 0.9062/0.47 & 0.5135/12.93 & 0.5111/12.53 & 0.5025/20.40 & 0.5033/18.88 & 0.9213/0.40 & 0.6898/2.27 \\
LibriSeVoc & 0.5817/9.66 & 0.5002/45.60 & 0.5017/43.91 & 0.5002/50.06 & 0.5003/47.18 & 0.5026/34.40 & 0.7632/2.08 \\
Ours & 0.4864/51.89 & 0.4914/50.00 & 0.4853/52.37 & 0.4979/50.17 & 0.4804/65.70 & 0.4925/46.07 & 0.6755/23.66 \\
RawNet2 & 0.6991/4.19 & 0.5008/33.67 & 0.5049/23.08 & 0.5003/40.52 & 0.5015/31.79 & 0.5301/15.51 & 0.9742/0.39 \\
Wav2Clip & 0.5047/1.02 & 0.5000/12.83 & 0.5000/10.73 & 0.5000/9.92 & 0.5000/8.59 & 0.7363/0.26 & 0.5010/1.43 \\
Wave2Vec2 & 0.5156/19.27 & 0.5008/35.84 & 0.5027/28.60 & 0.5001/42.81 & 0.5010/36.15 & 0.5040/26.69 & 0.6266/8.45 \\
WaveLM & 0.7255/10.97 & 0.5046/41.82 & 0.5418/25.21 & 0.5025/43.82 & 0.5161/35.01 & 0.59

## Cross Language

### WaveFake

In [11]:
DATA = []
for task in [
    "wavefake_cross_lang",
]:
    for model in MODELs:
        save_path = f"{ROOT_PATH}/{model}/{task}/version_0"
        csv_path = os.path.join(save_path, "metrics.csv")
        try:
            data = pd.read_csv(csv_path)
            data = data[["test-acc", "test-auc", "test-eer"]]
        except FileNotFoundError:
            continue
        except KeyError:
            continue
        res = data.dropna().reset_index(drop=True)
        res["res"] = res.apply(
            lambda x: "{:.4f}/{:.2f}".format(x["test-auc"], x["test-eer"] * 100),
            axis=1,
        )
        res["model"] = model
        res = res[["model", "res"]]
        DATA.append(res)
data = pd.concat(DATA)
data1 = data.set_index("model").sort_index()

print(data1.style.to_latex())
data1

\begin{tabular}{ll}
 & res \\
model &  \\
AASIST & 0.9010/7.56 \\
AudioClip & 0.5278/40.26 \\
LCNN & 0.6832/15.20 \\
LibriSeVoc & 0.5508/31.36 \\
Ours & 0.6825/20.02 \\
RawNet2 & 0.6139/35.07 \\
Wav2Clip & 0.5775/16.92 \\
Wave2Vec2 & 0.7682/21.70 \\
WaveLM & 0.5860/22.32 \\
\end{tabular}



,res
model,
AASIST,0.9010/7.56
AudioClip,0.5278/40.26
LCNN,0.6832/15.20
LibriSeVoc,0.5508/31.36
Ours,0.6825/20.02
RawNet2,0.6139/35.07
Wav2Clip,0.5775/16.92
Wave2Vec2,0.7682/21.70
WaveLM,0.5860/22.32


### DECRO

In [12]:
DATA = []
for task in ["DECRO_chinese", "DECRO_english"]:
    for model in MODELs:
        save_path = f"{ROOT_PATH}/{model}/{task}/version_0"
        csv_path = os.path.join(save_path, "metrics.csv")
        try:
            data = pd.read_csv(csv_path)
            data = data[["test-acc", "test-auc", "test-eer"]]
        except FileNotFoundError:
            continue
        except KeyError:
            continue
        res = data.dropna().reset_index(drop=True)
        res["res"] = res.apply(
            lambda x: "{:.4f}/{:.2f}".format(x["test-auc"], x["test-eer"] * 100),
            axis=1,
        )
        res["model"] = model
        res['language'] = task
        res = res[["model", "res", "language"]]
        DATA.append(res)
data = pd.concat(DATA)
# data = data.set_index("model").sort_index()
data2 = data.pivot(index='model', columns='language', values='res')
print(data2.style.to_latex())
data2

\begin{tabular}{lll}
language & DECRO_chinese & DECRO_english \\
model &  &  \\
AASIST & 0.6588/41.03 & 0.6937/40.90 \\
AudioClip & 0.6258/47.15 & 0.6440/44.85 \\
LCNN & 0.4730/45.18 & 0.6376/43.89 \\
LibriSeVoc & 0.5461/47.52 & 0.5305/50.97 \\
Ours & 0.7370/26.04 & 0.7292/26.50 \\
RawNet2 & 0.5277/51.30 & 0.4918/53.46 \\
Wav2Clip & 0.7125/23.13 & 0.7121/20.59 \\
Wave2Vec2 & 0.3766/53.14 & 0.3710/54.58 \\
WaveLM & 0.3202/58.24 & 0.3225/58.70 \\
\end{tabular}



language,DECRO_chinese,DECRO_english
model,,
AASIST,0.6588/41.03,0.6937/40.90
AudioClip,0.6258/47.15,0.6440/44.85
LCNN,0.4730/45.18,0.6376/43.89
LibriSeVoc,0.5461/47.52,0.5305/50.97
Ours,0.7370/26.04,0.7292/26.50
RawNet2,0.5277/51.30,0.4918/53.46
Wav2Clip,0.7125/23.13,0.7121/20.59
Wave2Vec2,0.3766/53.14,0.3710/54.58
WaveLM,0.3202/58.24,0.3225/58.70


## merge wavefake and DECRO data

In [13]:
data = data1.merge(data2, left_index=True, right_index=True)
print(data.style.to_latex())
data

\begin{tabular}{llll}
 & res & DECRO_chinese & DECRO_english \\
model &  &  &  \\
AASIST & 0.9010/7.56 & 0.6588/41.03 & 0.6937/40.90 \\
AudioClip & 0.5278/40.26 & 0.6258/47.15 & 0.6440/44.85 \\
LCNN & 0.6832/15.20 & 0.4730/45.18 & 0.6376/43.89 \\
LibriSeVoc & 0.5508/31.36 & 0.5461/47.52 & 0.5305/50.97 \\
Ours & 0.6825/20.02 & 0.7370/26.04 & 0.7292/26.50 \\
RawNet2 & 0.6139/35.07 & 0.5277/51.30 & 0.4918/53.46 \\
Wav2Clip & 0.5775/16.92 & 0.7125/23.13 & 0.7121/20.59 \\
Wave2Vec2 & 0.7682/21.70 & 0.3766/53.14 & 0.3710/54.58 \\
WaveLM & 0.5860/22.32 & 0.3202/58.24 & 0.3225/58.70 \\
\end{tabular}



,res,DECRO_chinese,DECRO_english
model,,,
AASIST,0.9010/7.56,0.6588/41.03,0.6937/40.90
AudioClip,0.5278/40.26,0.6258/47.15,0.6440/44.85
LCNN,0.6832/15.20,0.4730/45.18,0.6376/43.89
LibriSeVoc,0.5508/31.36,0.5461/47.52,0.5305/50.97
Ours,0.6825/20.02,0.7370/26.04,0.7292/26.50
RawNet2,0.6139/35.07,0.5277/51.30,0.4918/53.46
Wav2Clip,0.5775/16.92,0.7125/23.13,0.7121/20.59
Wave2Vec2,0.7682/21.70,0.3766/53.14,0.3710/54.58
WaveLM,0.5860/22.32,0.3202/58.24,0.3225/58.70
